# Sound Classification with CatBoost and hyperparameter tuning

In [1]:
import os
import numpy as np
import pandas as pd
import librosa as lb
import IPython.display as ipd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

In [2]:
metadata = pd.read_csv('C:/Users/user/PycharmProjects/pythonProject/CatSound/metadata/cat2.csv')
print(metadata.shape)
metadata.head()

(1120, 4)


,slice_file_name,fold,classID,class
0,purring (1).wav,1,0,purring
1,purring (2).wav,1,0,purring
2,purring (3).wav,1,0,purring
3,purring (4).wav,1,0,purring
4,purring (5).wav,1,0,purring


In [3]:
classes = metadata.groupby('classID')['class'].unique()
classes

classID
0     [purring]
1        [meow]
2    [grooming]
3    [drinking]
4          [wc]
5      [eating]
Name: class, dtype: object

# Featue Extraction

In [4]:
import librosa
import librosa.display

In [5]:
def feature_extractor(path):
    data, simple_rate = librosa.load(path)
    data = librosa.feature.mfcc(data,n_mfcc=128,sr=simple_rate)
    data = np.mean(data,axis=1)
    return data

In [ ]:
x, y = [], [] #feature extraction with mfcc
extracted_features=[]
for i,rows in tqdm(metadata.iterrows()):
    path = 'C:/Users/user/PycharmProjects/pythonProject/CatSound/audio/' + 'fold' + str(rows['fold']) + '/' + str(rows['slice_file_name'])
    x.append(feature_extractor(path))
    y.append(rows['classID'])
    fe = feature_extractor(path)
    final_class_labels=rows['class']
    extracted_features.append([fe,final_class_labels])
x = np.array(x)
y = np.array(y)
x.shape, y.shape

In [7]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-312.18478, 125.071014, -64.47501, 56.468628,...",purring
1,"[-408.58997, 146.62447, 32.391697, 36.671593, ...",purring
2,"[-440.81326, 190.86948, -21.095238, -6.7931995...",purring
3,"[-373.53268, 150.26164, 44.241188, 31.615002, ...",purring
4,"[-578.2753, 163.7805, 61.469208, 17.083605, -3...",purring


In [8]:
xtrainval, xtest, ytrainval, ytest = train_test_split(x,y,test_size=0.1,stratify=y,random_state=387)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrainval,ytrainval,test_size=0.2,stratify=ytrainval,random_state=387)
print('\nNumber of samples for Train set :',xtrain.shape[0])
print('Number of samples for Validation set :',xvalid.shape[0])
print('Number of samples for Test set :',xtest.shape[0])


Number of samples for Train set : 806
Number of samples for Validation set : 202
Number of samples for Test set : 112


In [9]:
xtrain.shape

(806, 128)

In [10]:
from catboost import CatBoostClassifier
from catboost import CatBoost
from catboost import Pool
from catboost import cv
from catboost.utils import eval_metric
from catboost import Pool
import catboost as cb

In [12]:
model = cb.CatBoostClassifier(loss_function='MultiClass', 
                              eval_metric='Accuracy')

In [13]:
grid = {'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations':[5,10,50],
        'learning_rate':[0.01, 0.03, 0.1,1.0],
        'bootstrap_type':['Bayesian', 'Bernoulli', 'MVS', 'No']}

In [15]:
train_data = Pool(xtrain, ytrain)
eval_data = Pool(xvalid, yvalid)

In [16]:
model.grid_search(grid,train_data,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7391304	test: 0.7283951	best: 0.7283951 (0)	total: 171ms	remaining: 682ms
1:	learn: 0.8338509	test: 0.8209877	best: 0.8209877 (1)	total: 178ms	remaining: 267ms
2:	learn: 0.8214286	test: 0.8148148	best: 0.8209877 (1)	total: 186ms	remaining: 124ms
3:	learn: 0.8555901	test: 0.8580247	best: 0.8580247 (3)	total: 193ms	remaining: 48.3ms
4:	learn: 0.8695652	test: 0.8765432	best: 0.8765432 (4)	total: 201ms	remaining: 0us

bestTest = 0.8765432099
bestIteration = 4

0:	loss: 0.8765432	best: 0.8765432 (0)	total: 250ms	remaining: 1m 47s
0:	learn: 0.6708075	test: 0.6543210	best: 0.6543210 (0)	total: 9.17ms	remaining: 36.7ms
1:	learn: 0.8183230	test: 0.8148148	best: 0.8148148 (1)	total: 16.4ms	remaining: 24.6ms
2:	learn: 0.8416149	test: 0.7962963	best: 0.8148148 (1)	total: 24.5ms	remaining: 16.3ms
3:	learn: 0.8742236	test: 0.8395062	best: 0.8395062 (3)	total: 32.4ms	remaining: 8.11ms
4:	learn: 0.8804348	test: 0.8765432	best: 0.8765432 (4)	total: 40.4ms	remaining: 0us

bestTest = 0.876543

1:	learn: 0.8338509	test: 0.8209877	best: 0.8209877 (1)	total: 27.5ms	remaining: 41.3ms
2:	learn: 0.8183230	test: 0.8148148	best: 0.8209877 (1)	total: 35.4ms	remaining: 23.6ms
3:	learn: 0.8447205	test: 0.8518519	best: 0.8518519 (3)	total: 43.6ms	remaining: 10.9ms
4:	learn: 0.8602484	test: 0.8641975	best: 0.8641975 (4)	total: 51.2ms	remaining: 0us

bestTest = 0.8641975309
bestIteration = 4

16:	loss: 0.8641975	best: 0.8950617 (12)	total: 1.22s	remaining: 29.7s
0:	learn: 0.6708075	test: 0.6543210	best: 0.6543210 (0)	total: 24.2ms	remaining: 96.7ms
1:	learn: 0.8183230	test: 0.8148148	best: 0.8148148 (1)	total: 31.5ms	remaining: 47.3ms
2:	learn: 0.8307453	test: 0.7962963	best: 0.8148148 (1)	total: 38.9ms	remaining: 25.9ms
3:	learn: 0.8695652	test: 0.8271605	best: 0.8271605 (3)	total: 46ms	remaining: 11.5ms
4:	learn: 0.8757764	test: 0.8580247	best: 0.8580247 (4)	total: 53.2ms	remaining: 0us

bestTest = 0.8580246914
bestIteration = 4

17:	loss: 0.8580247	best: 0.8950617 (12)	total: 1.28s	rem

0:	learn: 0.7826087	test: 0.8024691	best: 0.8024691 (0)	total: 27.1ms	remaining: 109ms
1:	learn: 0.8027950	test: 0.8148148	best: 0.8148148 (1)	total: 34.2ms	remaining: 51.3ms
2:	learn: 0.8633540	test: 0.8518519	best: 0.8518519 (2)	total: 41.2ms	remaining: 27.5ms
3:	learn: 0.8540373	test: 0.8333333	best: 0.8518519 (2)	total: 48.2ms	remaining: 12ms
4:	learn: 0.8664596	test: 0.8456790	best: 0.8518519 (2)	total: 55.5ms	remaining: 0us

bestTest = 0.8518518519
bestIteration = 2

35:	loss: 0.8518519	best: 0.8950617 (12)	total: 2.19s	remaining: 24.1s
0:	learn: 0.7391304	test: 0.7283951	best: 0.7283951 (0)	total: 11.6ms	remaining: 46.6ms
1:	learn: 0.8307453	test: 0.8148148	best: 0.8148148 (1)	total: 19ms	remaining: 28.5ms
2:	learn: 0.8167702	test: 0.8024691	best: 0.8148148 (1)	total: 26.3ms	remaining: 17.5ms
3:	learn: 0.8416149	test: 0.8518519	best: 0.8518519 (3)	total: 33.7ms	remaining: 8.43ms
4:	learn: 0.8586957	test: 0.8641975	best: 0.8641975 (4)	total: 41ms	remaining: 0us

bestTest = 0.8641

1:	learn: 0.7670807	test: 0.7716049	best: 0.7716049 (1)	total: 42.4ms	remaining: 170ms
2:	learn: 0.8152174	test: 0.8209877	best: 0.8209877 (2)	total: 49.9ms	remaining: 116ms
3:	learn: 0.8804348	test: 0.8580247	best: 0.8580247 (3)	total: 57.8ms	remaining: 86.7ms
4:	learn: 0.9021739	test: 0.8827160	best: 0.8827160 (4)	total: 65.4ms	remaining: 65.4ms
5:	learn: 0.8990683	test: 0.8950617	best: 0.8950617 (5)	total: 72.9ms	remaining: 48.6ms
6:	learn: 0.9052795	test: 0.9012346	best: 0.9012346 (6)	total: 80.3ms	remaining: 34.4ms
7:	learn: 0.9068323	test: 0.9074074	best: 0.9074074 (7)	total: 87.7ms	remaining: 21.9ms
8:	learn: 0.9052795	test: 0.9012346	best: 0.9074074 (7)	total: 95.2ms	remaining: 10.6ms
9:	learn: 0.9099379	test: 0.9074074	best: 0.9074074 (7)	total: 103ms	remaining: 0us

bestTest = 0.9074074074
bestIteration = 7

50:	loss: 0.9074074	best: 0.9074074 (48)	total: 3.19s	remaining: 23.8s
0:	learn: 0.7826087	test: 0.8024691	best: 0.8024691 (0)	total: 33.4ms	remaining: 301ms
1:	learn: 0.

...

43:	learn: 0.9984472	test: 0.9320988	best: 0.9444444 (25)	total: 41.6s	remaining: 5.67s
44:	learn: 0.9984472	test: 0.9320988	best: 0.9444444 (25)	total: 42.5s	remaining: 4.72s
45:	learn: 0.9984472	test: 0.9382716	best: 0.9444444 (25)	total: 43.4s	remaining: 3.78s
46:	learn: 0.9984472	test: 0.9382716	best: 0.9444444 (25)	total: 44.4s	remaining: 2.83s
47:	learn: 0.9984472	test: 0.9382716	best: 0.9444444 (25)	total: 45.3s	remaining: 1.89s
48:	learn: 0.9984472	test: 0.9444444	best: 0.9444444 (25)	total: 46.3s	remaining: 944ms
49:	learn: 0.9984472	test: 0.9444444	best: 0.9444444 (25)	total: 47.2s	remaining: 0us

bestTest = 0.9444444444
bestIteration = 25

425:	loss: 0.9444444	best: 0.9876543 (254)	total: 47m 19s	remaining: 40s
0:	learn: 0.9363354	test: 0.7530864	best: 0.7530864 (0)	total: 917ms	remaining: 45s
1:	learn: 0.9565217	test: 0.8580247	best: 0.8580247 (1)	total: 1.92s	remaining: 46.1s
2:	learn: 0.9549689	test: 0.9074074	best: 0.9074074 (2)	total: 2.92s	remaining: 45.7s
3:	learn: 0.

35:	learn: 0.9953416	test: 0.9506173	best: 0.9567901 (34)	total: 35.3s	remaining: 13.7s
36:	learn: 0.9953416	test: 0.9506173	best: 0.9567901 (34)	total: 36.3s	remaining: 12.8s
37:	learn: 0.9953416	test: 0.9506173	best: 0.9567901 (34)	total: 37.3s	remaining: 11.8s
38:	learn: 0.9968944	test: 0.9444444	best: 0.9567901 (34)	total: 38.3s	remaining: 10.8s
39:	learn: 0.9968944	test: 0.9567901	best: 0.9567901 (34)	total: 39.2s	remaining: 9.81s
40:	learn: 0.9953416	test: 0.9506173	best: 0.9567901 (34)	total: 40.2s	remaining: 8.83s
41:	learn: 0.9968944	test: 0.9506173	best: 0.9567901 (34)	total: 41.2s	remaining: 7.85s
42:	learn: 0.9953416	test: 0.9506173	best: 0.9567901 (34)	total: 42.2s	remaining: 6.87s
43:	learn: 0.9968944	test: 0.9506173	best: 0.9567901 (34)	total: 43.1s	remaining: 5.88s
44:	learn: 0.9968944	test: 0.9567901	best: 0.9567901 (34)	total: 44.1s	remaining: 4.9s
45:	learn: 0.9968944	test: 0.9567901	best: 0.9567901 (34)	total: 45.1s	remaining: 3.92s
46:	learn: 0.9968944	test: 0.9567

27:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 27.4s	remaining: 21.6s
28:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 28.4s	remaining: 20.6s
29:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 29.3s	remaining: 19.6s
30:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 30.3s	remaining: 18.5s
31:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 31.3s	remaining: 17.6s
32:	learn: 1.0000000	test: 0.9506173	best: 0.9506173 (25)	total: 32.2s	remaining: 16.6s
33:	learn: 1.0000000	test: 0.9567901	best: 0.9567901 (33)	total: 33.2s	remaining: 15.6s
34:	learn: 1.0000000	test: 0.9567901	best: 0.9567901 (33)	total: 34.2s	remaining: 14.7s
35:	learn: 1.0000000	test: 0.9629630	best: 0.9629630 (35)	total: 35.2s	remaining: 13.7s
36:	learn: 1.0000000	test: 0.9629630	best: 0.9629630 (35)	total: 36.1s	remaining: 12.7s
37:	learn: 1.0000000	test: 0.9629630	best: 0.9629630 (35)	total: 37.1s	remaining: 11.7s
38:	learn: 1.0000000	test: 0.962

19:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 19.4s	remaining: 29.1s
20:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 20.4s	remaining: 28.2s
21:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 21.4s	remaining: 27.2s
22:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 22.4s	remaining: 26.3s
23:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 23.4s	remaining: 25.3s
24:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 24.3s	remaining: 24.3s
25:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 25.3s	remaining: 23.4s
26:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 26.3s	remaining: 22.4s
27:	learn: 1.0000000	test: 0.9320988	best: 0.9320988 (8)	total: 27.3s	remaining: 21.4s
28:	learn: 1.0000000	test: 0.9382716	best: 0.9382716 (28)	total: 28.2s	remaining: 20.4s
29:	learn: 1.0000000	test: 0.9382716	best: 0.9382716 (28)	total: 29.2s	remaining: 19.4s
30:	learn: 1.0000000	test: 0.9382716	best

{'params': {'depth': 6,
  'l2_leaf_reg': 1,
  'iterations': 50,
  'learning_rate': 1.0,
  'bootstrap_type': 'MVS'},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
           

In [17]:
history = model.fit(train_data, 
          eval_set=eval_data,
          early_stopping_rounds=50,
          plot=True,
          silent=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8163772	test: 0.7722772	best: 0.7722772 (0)	total: 37ms	remaining: 1.81s
1:	learn: 0.8883375	test: 0.8019802	best: 0.8019802 (1)	total: 69.3ms	remaining: 1.66s
2:	learn: 0.9193548	test: 0.8613861	best: 0.8613861 (2)	total: 104ms	remaining: 1.62s
3:	learn: 0.9478908	test: 0.8861386	best: 0.8861386 (3)	total: 137ms	remaining: 1.57s
4:	learn: 0.9677419	test: 0.9059406	best: 0.9059406 (4)	total: 169ms	remaining: 1.52s
5:	learn: 0.9801489	test: 0.8861386	best: 0.9059406 (4)	total: 202ms	remaining: 1.48s
6:	learn: 0.9950372	test: 0.9009901	best: 0.9059406 (4)	total: 236ms	remaining: 1.45s
7:	learn: 0.9962779	test: 0.9306931	best: 0.9306931 (7)	total: 268ms	remaining: 1.41s
8:	learn: 0.9987593	test: 0.9405941	best: 0.9405941 (8)	total: 301ms	remaining: 1.37s
9:	learn: 0.9987593	test: 0.9455446	best: 0.9455446 (9)	total: 334ms	remaining: 1.33s
10:	learn: 1.0000000	test: 0.9554455	best: 0.9554455 (10)	total: 367ms	remaining: 1.3s
11:	learn: 1.0000000	test: 0.9504950	best: 0.9554455 

In [27]:
print("\n The best score across ALL searched params:\n", model.best_score_)


 The best score across ALL searched params:
 {'learn': {'Accuracy': 1.0, 'MultiClass': 0.002529426432783263}, 'validation': {'Accuracy': 0.9851485148514851, 'MultiClass': 0.05290394583637222}}


In [29]:
from sklearn.metrics import confusion_matrix

In [39]:
ytrue = (ytest)
ypred = (model.predict(xtest))
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(ytrue,ypred))
print('\n\nClassification Report : \n\n',classification_report(ytrue,ypred))


Confusion Matrix :


[[19  0  1  0  0  0]
 [ 0 19  0  0  0  1]
 [ 0  0 20  0  0  0]
 [ 0  0  0 19  0  1]
 [ 0  0  0  0 11  1]
 [ 0  0  0  0  0 20]]


Classification Report : 

               precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       1.00      0.95      0.97        20
           2       0.95      1.00      0.98        20
           3       1.00      0.95      0.97        20
           4       1.00      0.92      0.96        12
           5       0.87      1.00      0.93        20

    accuracy                           0.96       112
   macro avg       0.97      0.96      0.96       112
weighted avg       0.97      0.96      0.96       112



In [75]:
model.save_model("model.py")

In [53]:
model.get_params()

{'loss_function': 'MultiClass',
 'eval_metric': 'Accuracy',
 'depth': 6,
 'l2_leaf_reg': 1,
 'iterations': 50,
 'learning_rate': 1.0,
 'bootstrap_type': 'MVS'}